# Seskupování dat (Grouping data)

Seskupování dat je jednou ze základních analytických operací. Umožňuje nám vyvozovat zobecněné závěry na základě množiny dat.

Základní agregační funkce:
- **mean** - průměr
- **median** - medián (střední hodnota)
- **min** - minimum
- **max** - maximum
- **count** - počet pozorování

## Příprava dat

V této lekci budeme pracovat s vyčištěnými daty ze souboru `product_prices_cleaned.csv`.

In [ ]:
# Import knihovny pandas
import pandas as pd

In [ ]:
# Načtení dat
df = pd.read_csv(r'..\Data\product_prices_renamed.csv', sep = ';')

In [ ]:
# Zobrazení prvních řádků
df.head()

In [ ]:
# Zjištění informací o datech
df.info()

---

## Základní agregační funkce v Pandas

| Statistika | Název v Pandas | Příklad použití |
|------------|----------------|------------------|
| průměr | mean | `df['value'].mean()` |
| medián | median | `df['value'].median()` |
| minimum | min | `df['value'].min()` |
| maximum | max | `df['value'].max()` |
| počet | count | `df['value'].count()` |

In [ ]:
# Průměrná cena
df['value'].mean()

In [ ]:
# Medián ceny
df['value'].median()

In [ ]:
# Minimální cena
df['value'].min()

In [ ]:
# Maximální cena
df['value'].max()

In [ ]:
# Počet hodnot
df['value'].count()

---

## Seskupování v Pandas - metoda `groupby`

Klíčem k datové analýze je správné rozdělení dat - abychom mohli vyvozovat správné závěry. Příslovečné "porovnávání jablek s hruškami" se týká právě tohoto problému.

Pandas nabízí flexibilní rozhraní `groupby`, které umožňuje přirozeně rozdělovat a sumarizovat datové sady.

### Jednoduchý příklad na malých datech

Nejprve si ukážeme `groupby` na jednoduchém příkladu, abychom pochopili princip.

In [ ]:
# Vytvoření malého DataFrame pro ukázku
data = {
    'produkt': ['jablka', 'jablka', 'hrušky', 'hrušky', 'banány', 'banány'],
    'obchod': ['Tesco', 'Billa', 'Tesco', 'Billa', 'Tesco', 'Billa'],
    'cena': [25, 28, 30, 32, 20, 22]
}

df_ovoce = pd.DataFrame(data)
df_ovoce

In [ ]:
# Seskupení podle produktu
skupiny_podle_produktu = df_ovoce.groupby('produkt')
print(type(skupiny_podle_produktu))

In [ ]:
# Jaké máme skupiny?
skupiny_podle_produktu.groups.keys()

In [ ]:
# Průměrná cena každého produktu
skupiny_podle_produktu['cena'].mean()

In [ ]:
# Iterace přes skupiny - podívejme se, co je v každé skupině
for produkt, data in skupiny_podle_produktu:
    print(f"\n=== {produkt} ===")
    print(data)

In [ ]:
# Výběr konkrétní skupiny
skupiny_podle_produktu.get_group('jablka')

In [ ]:
# Více statistik najednou pomocí agg
skupiny_podle_produktu['cena'].agg(['min', 'max', 'mean'])

**Co se děje:**
1. `groupby('produkt')` rozdělí data do skupin podle hodnot ve sloupci 'produkt'
2. Každá skupina obsahuje všechny řádky se stejnou hodnotou produktu
3. Na každou skupinu můžeme aplikovat agregační funkce (mean, min, max...)

---

## Práce s reálnými daty

Nyní použijeme stejné principy na větším datasetu s cenami produktů.

### Základní použití `groupby`

Metoda `groupby` seskupuje DataFrame podle jednoho nebo více sloupců. Vrací objekt `DataFrameGroupBy`.

In [ ]:
# Seskupení podle data
df_by_date = df.groupby('date')

In [ ]:
# Vrací objekt DataFrameGroupBy
type(df_by_date)

Objekt `DataFrameGroupBy` si můžeme představit jako kolekci DataFrame, kde každý obsahuje řádky se stejnou hodnotou ve sloupci, podle kterého jsme seskupovali.

---

## Iterace přes skupiny

Při rozbalení objektu `DataFrameGroupBy` dostaneme dvě proměnné:
1. **key** - unikátní hodnota ze sloupce, podle kterého seskupujeme
2. **podmnožina dat** - řádky se stejnou hodnotou klíče (je to DataFrame)

In [ ]:
# Iterace přes první 3 skupiny (pro ukázku)
counter = 0
for (key, df_date) in df_by_date:
    print(f"Klíč: {key}")
    print(df_date.head(3))
    print("-" * 50)
    counter += 1
    if counter >= 3:
        break

### Práce s konkrétními skupinami

Užitečné metody:
- `groups.keys()` - zobrazí klíče, podle kterých byla kolekce rozdělena
- `get_group()` - vybere konkrétní podmnožinu

In [ ]:
# Zobrazení dostupných skupin (klíčů)
df_by_date.groups.keys()

In [ ]:
# Výběr konkrétní skupiny - například '2019-9-01'
# (upravte datum podle dat ve vašem souboru)
# df_by_date.get_group('2019-9-01')

**Otázka k zamyšlení:** Proč je výsledek `get_group()` typu DataFrame a ne DataFrameGroupBy?

---

## Statistiky na seskupených datech

Objekt `DataFrameGroupBy` implementuje všechny základní statistiky jako `DataFrame`. Rozdíl je v tom, že statistiky se počítají pro každou skupinu zvlášť.

In [ ]:
# Maximální cena v daném měsíci
df_by_date['value'].max()

In [ ]:
# Průměrná cena produktů v měsíci
df_by_date['value'].mean()

**Úloha - oprav chybu:** Následující kód obsahuje chybu. Najdi ji a oprav.

In [ ]:
# Chyba: Chceme spočítat počet záznamů pro každé datum
# df_by_date.count['value']()

---

## Funkce `agg` / `aggregate`

Funkce `agg` slouží k agregaci řádků nebo sloupců pomocí jedné nebo více funkcí. Můžeme použít:
- předdefinované funkce (`mean`, `median`, `min`, `max`...)
- vlastní funkce
- lambda funkce

S `agg` můžeme analyzovat každý sloupec nezávisle.

**Poznámka:** Funkce `aggregate` je totožná s `agg` - je to alias.

In [ ]:
# Určení maximální, minimální a průměrné ceny ve sloupci value
df[['value']].agg(["min", "max", "mean"])

### Kombinace `groupby` a `agg`

Nejčastěji se `agg` používá ve spojení s `groupby` pro výpočet více statistik najednou.

In [ ]:
# Více statistik pro každou skupinu
df.groupby('date')['value'].agg(['min', 'max', 'mean', 'median', 'std'])

**Otázka k zamyšlení:** Co znamená `std` ve výstupu? Kdy je tato statistika užitečná?

### Seskupení podle více sloupců

In [ ]:
# Seskupení podle produktu a data
df.groupby(['product_types', 'date'])['value'].mean()

**Úloha - doplň kód:** Doplň chybějící část kódu pro výpočet minimální a maximální ceny pro každý produkt.

In [ ]:
# Doplň: min a max cena pro každý produkt
# df.groupby(___)[___].agg([___, ___])

---

## Praktické cvičení

**Úloha - oprav chybu:** V následujícím kódu je syntaktická chyba. Oprav ji.

In [ ]:
# Chyba v kódu
# df.groupby('product').['value'].mean()

**Úloha - doplň kód:** Iterace přes skupiny - doplň chybějící části.

In [ ]:
# Doplň proměnné v cyklu for
# df_by_product = df.groupby('product')
# for (___, ___) in df_by_product:
#     print(f"Produkt: {___}")
#     print(f"Průměrná cena: {___['value'].mean()}")

---

## Úlohy k procvičení

### Úloha 1: Seskupování produktů

Pomocí dat ze souboru **product_prices_cleaned.csv** vyřeš následující úkoly:

1. Jaká byla průměrná měsíční cena každé komodity?
2. Který produkt měl nejvyšší cenovou volatilitu (směrodatnou odchylku) za celé období?

Použij sloupce **product** a **value** pro analýzu.

Dodatečné otázky:
- Jsou potřeba nějaké další předpoklady pro tyto úlohy?
- Proč lze tuto úlohu provést až teď, po vyčištění dat?

In [ ]:
# Řešení úlohy 1.1 - průměrná měsíční cena každé komodity


In [ ]:
# Řešení úlohy 1.2 - produkt s nejvyšší volatilitou


### Úloha 2: Agregace

Pomocí dat ze souboru **product_prices_cleaned.csv** proveď agregaci dat pro každý produkt podle měsíce a urči statistiky: `min, max, median, mean, std` pro ceny (sloupec **value**):

1. Vynech národní data z analýzy
2. Proveď agregaci přímo na objektu z `groupby`
3. Napiš cyklus, který spočítá tyto hodnoty pro jednotlivé provincie

Použij metodu `agg` a seskup data podle sloupců `'product', 'date'`.

In [ ]:
# Řešení úlohy 2.1 - vynechání národních dat


In [ ]:
# Řešení úlohy 2.2 - agregace s groupby


In [ ]:
# Řešení úlohy 2.3 - cyklus pro jednotlivé provincie


---

## Přehled použitých metod a funkcí

| Metoda/Funkce | Použití |
|---------------|--------|
| `df.groupby(sloupec)` | Seskupí DataFrame podle zadaného sloupce/sloupců |
| `df.groupby([sloupec1, sloupec2])` | Seskupení podle více sloupců |
| `grouped.groups.keys()` | Zobrazí všechny klíče (unikátní hodnoty) skupin |
| `grouped.get_group(hodnota)` | Vrátí DataFrame pro konkrétní skupinu |
| `df['sloupec'].mean()` | Spočítá průměr hodnot ve sloupci |
| `df['sloupec'].median()` | Spočítá medián hodnot ve sloupci |
| `df['sloupec'].min()` | Vrátí minimální hodnotu ve sloupci |
| `df['sloupec'].max()` | Vrátí maximální hodnotu ve sloupci |
| `df['sloupec'].count()` | Spočítá počet neprázdných hodnot |
| `df['sloupec'].std()` | Spočítá směrodatnou odchylku |
| `df.agg([funkce1, funkce2])` | Aplikuje více agregačních funkcí najednou |
| `grouped['sloupec'].agg([...])` | Agregace na seskupených datech |